<a href="https://colab.research.google.com/github/mlindsey19/evol-comp-two/blob/master/EvolComp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

N = 10 # must be even for crossover
RUNS = 30
LOCI = 3
LCBD = (-1.0, 5.0) #loci bounds
#print format -> #.###
float_formatter = lambda x: "%.3f" % x
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )


In [51]:
!git status

On branch master

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [0]:
  gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
  
  print (gen)

In [0]:
pen = 2
penFlag = 1

def fitness( individual ):
  fit = 0
  for x in individual:
    fit +=  pen * x**2 if ( ( x > 5.0 or x < -1.0 ) and penFlag ) else x**2
  return fit

def fitVector(g, n):
  fit = np.empty(n)
  for i in range(n):
    fit[ i ] = fitness( g[ i ] )
  return fit


def sortFit( g, n = N ):
  temp = np.c_[ g, fitVector( g, n ) ]
  return temp[ temp[ : , 3 ].argsort() ]


In [0]:
Pc = 0.8
alpha = 0.7
shuffFlag = 1

def crossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      gen[ j ][ 0 ], gen[ j + 1 ][ 0 ] = gen[ j + 1 ][ 0 ],  gen[ j ][ 0 ]
#  return gen

def arithmeticCrossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      x = gen[ j + 1 ][ 0 ] + gen[ j ][ 0 ]
      gen[ j ][ 0 ], gen[ j + 1 ][ 0 ] = ( 1 - alpha ) * x, alpha * x
  #    return gen

def arithmeticCrossPerLoci(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    for h in range( 3 ):        
      j = 2 * i
      if ( np.random.uniform( 0, 1 ) < Pc ):
        x = gen[ j + 1 ][ h ] + gen[ j ][ h ]
        gen[ j ][ h ], gen[ j + 1 ][ h ] = ( 1 - alpha ) * x, alpha * x
 # return gen
    
  
 

***mr*** mutation range
***ub*** upper bound

The upper bound is calculated so that numbers selected at random fall in an appropriate 
range of mutation ***P*** % of the time 

In [0]:
Pm = 0.1
mr = 1.75

def mut():
  mtmt = np.empty( ( N, LOCI) )
  ub = ( 2 * mr / Pm ) - mr
  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( (0 - mr), ub )
      if ( m < mr ):
        mtmt[i][j] = m
      else:
        mtmt[i][j] = 0  
 # print (mtmt)
  return mtmt

def mutation(  ):
  global gen
  gen = gen + mut()
 # print (mut().shape)
  #print (gen.shape)
 # return mut() + gen
  

def mutationWithClamp(  ):
  global gen
#  print(gen)
  ngen = gen + mut()
  for rw in range( N ):
    for cl in range( LOCI ):  
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] > -1 ) else -1
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] < 5 ) else 5
  gen = ngen

  
mutation()
mutationWithClamp()

print( gen )



In [0]:
Psbt = 0.4

def selectionBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    #print( x , y)
    if( fitness( x ) > fitness( y ) ):
      x = y
    ngen[ i ] = x
  gen = ngen

selectionBT()
print(gen)

In [0]:
min = .5
max = 2 - min
Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
pmap =np.empty(N)
for i in range(N):
  if (i == 0):
    pmap[ i ] = PlrMap[ i ]
  else:
    pmap[ i ] = PlrMap[ i ] + pmap[ i - 1 ]


def selectionLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

en = 2  # number of best fit guarantee propogate  

def selectionEliteLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( en ):
    ngen[ i ] = temp[ i, : 3]
  for i in range( en, N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

selectionLR()
print(gen)
selectionEliteLR()

print( gen )

In [0]:
tn = 5 # number to keep

firstRunFlag = 1
if ( firstRunFlag ):
  firstRunFlag = 0
  prevgen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
  
def selectionTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  temp = sortFit( gen )
  for i in range( tn ):
    ngen[ i ] = temp[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

def selectionFibTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  global prevgen
  temp = gen
  union = np.append(gen, prevgen, axis = 0)
  union = sortFit(union, n=(2 * N) )
  prevgen = temp
  for i in range( tn ):
    ngen[ i ] = union[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

selectionFibTrunc()
print(gen)
#print (selectionTrunc( generation ) )

In [0]:
for i in range(50):
  mutation()
  mutationWithClamp()
  arithmeticCrossPerLoci()
  arithmeticCrossover()
  arithmeticCrossPerLoci()
  crossover()
  selectionBT() 
  selectionTrunc() 
  selectionFibTrunc()
  selectionLR()
  selectionEliteLR()
print (gen)



In [0]:


mutList = [ mutationWithClamp, mutation ]
crossList = [ arithmeticCrossover, arithmeticCrossPerLoci, crossover ]
selList = [ selectionBT, selectionTrunc, selectionFibTrunc, selectionLR, selectionEliteLR ]
def evol():
  avg = np.empty((5,3,2))
  best = np.empty((5,3,2))
  for i in range( 30 ):
    for (s, sel) in enumerate(selList):
      for c, cross in enumerate(crossList):
        for m, mut in enumerate(mutList):
          gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
          for j in range( 50 ):
              mut()
              cross()
              sel()
            avg[s][c][m] = np.mean(fitVector(gen))
            bst =  np.amin(fitVector(gen, N))
            best[s][c][m] = bst if (bst < best[s][c][m] ) else best[s][c][m]
          avg[s][c][m] /= 30
          
  print (avg)
  print(best)

evol()